In [ ]:
import Pkg
pkgs = [
    "Revise",
    "DataFrames",
    "uCSV",
    "Random",
    "ProgressMeter",
    "FASTX",
]
# Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end
import Mycelia

In [ ]:
project_dir = dirname(pwd())
data_dir = mkpath(joinpath(project_dir, "data"))
# https://ictv.global/vmr/current

In [ ]:
readdir(data_dir)

In [ ]:
ictv_vmr_file = joinpath(data_dir, "VMR_MSL38_v2.tsv")

In [ ]:
ictv_table = DataFrames.DataFrame(uCSV.read(ictv_vmr_file, delim='\t', header=1, typedetectrows=100))

In [ ]:
outdir = mkpath(joinpath(data_dir, "ictv_genomes"))

In [ ]:
accessions = reduce(vcat, map(x -> strip.(last.(split.(split(x, ';'), ':'))), unique(filter(!isempty, ictv_table[!, "Virus GENBANK accession"]))))
accessions = unique(first.(split.(accessions)))

In [ ]:
ProgressMeter.@showprogress for accession in Random.shuffle(accessions)
    outfile = joinpath(outdir, accession * ".fna")
    if !isfile(outfile) || (filesize(outfile) == 0)
        try
            fasta_records = collect(Mycelia.get_sequence(db = "nuccore", accession = accession))
            open(outfile, "w") do io
                fastx_io = FASTX.FASTA.Writer(io)
                for fasta_record in fasta_records
                    write(fastx_io, fasta_record)
                end
                close(fastx_io)
            end
        catch e
            println("An error occurred: ", e)
        end
    end
end